Django проверяет данные в несколько этапов:

1. **Проверка каждого поля на соответствие заданному типу данных.** 
Любые параметры в запросе передаются в виде строк, а Django проверяет, что полученную строку можно привести к заданному типу — например к формату числа или даты.

2. **Проверка каждого поля штатными или кастомными функциями-валидаторами.** 
Например, можно проверить, что полученная в запросе дата находится в пределах какого-то диапазона или что значение целого числа не больше и не меньше указанного порога.

3. **Проверка и «очистка» значений полей специальными методами формы.**
Названия таких методов составляются из слова `clean` и названия поля: `clean_first_name()` например. Помимо валидации в этих полях может происходить и очистка данных, отсюда и название: to clean означает «чистить, очищать». Например, из поля, где ожидается название города, можно убирать все сокращения: город, г., гор., чтобы на выходе получить чистое название города.

4. **Проверка нескольких полей на совместимость.**
В форме могут быть взаимозависимые поля: например, при покупке авиабилетов по маршруту туда-обратно дата полёта «туда» должна быть раньше даты «обратно», и эту зависимость необходимо проверить. Такие проверки выполняются в методе формы `clean()`.

Каждую из этих проверок можно настроить. Полный перечень этапов валидации данных есть [в документации](https://docs.djangoproject.com/en/3.2/ref/forms/validation/).

***
## Кастомный валидатор

Напишем валидатор для поля с датой рождения: будем проверять, что человек указал реалистичную дату дня рождения, то есть ему от 1 года до, скажем, 120 лет.

Работа с валидатором строится так:

* описывается функция-валидатор;

* валидатор подключается к форме: имя валидатора указывается в поле формы (для `forms.Form`) или в поле модели, к которой привязана форма (для `forms.ModelForm`).
Валидаторы, как правило, описывают в отдельном файле: это позволяет импортировать их в разные поля, формы и модели. 

Создайте файл *validators.py* в приложении **birthday**, в этом файле опишем валидатор `real_age()`. На вход этот валидатор будет принимать один параметр — значение поля c датой рождения, назовём его `value`. 

Валидатор не должен возвращать никаких значений: он только проверяет, что переданное в него значение соответствует заданным условиям. Если же проверка не пройдена — валидатор должен вызывать ошибку `django.core.exceptions.ValidationError`. 

```py
# birthday/validators.py
# Импортируем класс для работы с датами.
from datetime import date

# Импортируем ошибку валидации.
from django.core.exceptions import ValidationError


# На вход функция будет принимать дату рождения.
def real_age(value: date) -> None:
    # Считаем разницу между сегодняшним днём и днём рождения в днях 
    # и делим на 365.
    age = (date.today() - value).days / 365
    # Если возраст меньше 1 года или больше 120 лет — выбрасываем ошибку валидации.
    if age < 1 or age > 120:
        raise ValidationError(
            'Ожидается возраст от 1 года до 120 лет'
        )
```

Теперь нужно применить валидатор к форме. Если это форма `forms.Form`, то валидатор указывается в том поле формы, которое он должен проверять, в аргументе `validators`:

```py
# birthday/forms.py
from django import forms

# Импортируем функцию-валидатор.
from .validators import real_age


class BirthdayForm(forms.Form):
    first_name = forms.CharField(label='Имя', max_length=20)
    last_name = forms.CharField(
        label='Фамилия', required=False, help_text='Необязательное поле'
    )
    birthday = forms.DateField(
        label='Дата рождения',
        widget=forms.DateInput(attrs={'type': 'date'}),
        # В аргументе validators указываем список или кортеж 
        # валидаторов этого поля (валидаторов может быть несколько).
        validators=(real_age,),
    )
```

Для форм на основе `forms.ModelForm` валидатор указывается в поле самой модели — тоже в аргументе `validators`:

```py
# birthday/models.py
from django.db import models

# Импортируется функция-валидатор.
from .validators import real_age


class Birthday(models.Model):
    first_name = models.CharField('Имя', max_length=20)
    last_name = models.CharField(
        'Фамилия', max_length=20, help_text='Необязательное поле', blank=True
    )
    # Валидатор указывается в описании поля.
    birthday = models.DateField('Дата рождения', validators=(real_age,)) 
```



Если отправить через форму запрос, в котором возраст не укладывается в установленные рамки, то появится ошибка валидации:

![alt text](https://pictures.s3.yandex.net/resources/173_1682599500.png)

Если валидатор привязан к модели — валидация будет срабатывать и при управлении данными через формы в админке:

![alt text](https://pictures.s3.yandex.net/resources/174_1682599511.png)


***
## Метод clean для полей

Во многих культурах используются многосоставные имена, например — Жан-Клод Камиль Франсуа, Стефани Джоанн Анджелина или Хуан Антонио. Усовершенствуем приложение **birthday**: если пользователь вводит в поле `first_name` несколько имён — будем сохранять только первое — например, «Хуан» вместо «Хуан Антонио». 

Для дополнительной валидации данных и для их анализа и очистки применяются clean-методы полей формы. Эти методы ничего не принимают на вход: они берут значения из словаря `cleaned_data` (он доступен в экземпляре формы `self`). 

Имена clean-методов составляются по принципу `clean_<имя_поля>`.

В отличие от валидаторов **clean-методы** обязаны возвращать какое-то значение — старое, если в методе проводилась валидация, или новое — в случае, если метод очищал данные. Значение, которое возвращает clean-метод, записывается в словарь `cleaned_data`, заменяя прежнее значение.

```py
# birthday/forms.py
...
class BirthdayForm(forms.ModelForm):
    
    class Meta:
        ... 

    def clean_first_name(self):
        # Получаем значение имени из словаря очищенных данных.
        first_name = self.cleaned_data['first_name']
        # Разбиваем полученную строку по пробелам 
        # и возвращаем только первое имя.
        return first_name.split()[0] 
```

В строку с приветствием выводится только первая часть имени — именно так имя сохранено в БД; однако в форме осталось полное имя: причина в том, что в неё выводятся данные из словаря `request.POST` — значения, полученные из запроса, а не из сохранённого в БД объекта.

***
## Метод clean() для формы

Напишем ещё один валидатор — он будет проверять взаимную зависимость полей. Установим специальную проверку: пользователи сайта не должны выдавать себя за участников группы The Beatles, так что в полях «Имя» и «Фамилия» не должны отправляться пары значений «Джон»+«Леннон», «Пол»+«Маккартни», «Джордж»+«Харрисон» или «Ринго»+«Старр».

Добавьте в код формы `BirthdayForm` описание метода `clean()`:

```py
# birthday/forms.py
from django import forms
# Импортируем класс ошибки валидации.
from django.core.exceptions import ValidationError

from .models import Birthday

# Множество с именами участников Ливерпульской четвёрки.
BEATLES = {'Джон Леннон', 'Пол Маккартни', 'Джордж Харрисон', 'Ринго Старр'}


class BirthdayForm(forms.ModelForm):

    class Meta:    
        ...

    def clean_first_name(self):
        ...
    
    def clean(self):
        # Получаем имя и фамилию из очищенных полей формы.
        first_name = self.cleaned_data['first_name']
        last_name = self.cleaned_data['last_name']
        # Проверяем вхождение сочетания имени и фамилии во множество имён.
        if f'{first_name} {last_name}' in BEATLES:
            raise ValidationError(
                'Мы тоже любим Битлз, но введите, пожалуйста, настоящее имя!'
            ) 
```

При необходимости метод `clean()` может возвращать обновлённый словарь `cleaned_data`, но в нашей ситуации проводится только проверка данных, так что возвращать ничего не нужно.

Если класс формы унаследован от другого пользовательского класса формы, то в методе `clean()` первым делом нужно вызвать метод `super().clean()` родительского класса, чтобы прошла валидация родительского класса. Вызывать `super().clean()` необходимо и для форм на основе моделей, в таком случае форма проверит требования к уникальности полей или сочетанию полей, если они описаны. 

Добавим к модели `Birthday` проверку на уникальность записи: совокупность значений полей «Имя», «Фамилия» и «Дата рождения» не должна повторяться в БД. 

Подобные проверки настраиваются с помощью атрибута `constraints` (англ. «ограничения») подкласса `Meta`, где указывается класс `model.UniqueConstraint` (ограничение на уникальность). 

В этом классе указывается

* перечень полей, совокупность которых должна быть уникальна;

* имя ограничения.

```py
# birthday/models.py
...


class Birthday(models.Model):
    ...  # Список полей.

    class Meta:
        constraints = (
            models.UniqueConstraint(
                fields=('first_name', 'last_name', 'birthday'),
                name='Unique person constraint',
            ),
        )
```
 

Если просто описать ограничение в модели и не создавать миграции, то эта проверка сработает в админке, но не сработает в форме на странице *birthday/*. 

Чтобы проверка сработала и в форме `BirthdayForm`, надо вызвать родительский метод `super().clean()` в методе `clean()` формы.

```py
# birthday/forms.py
...

class BirthdayForm(forms.ModelForm):
    ...

    def clean(self):
        # Вызов родительского метода clean.
        super().clean()
        first_name = self.cleaned_data['first_name']
        last_name = self.cleaned_data['last_name']
        if f'{first_name} {last_name}' in BEATLES:
            raise ValidationError(
                'Мы тоже любим Битлз, но введите, пожалуйста, настоящее имя!'
            )
```
 
После этого ограничение заработает — и создать две одинаковые записи будет невозможно.

![alt text](https://pictures.s3.yandex.net/resources/177_1682599602.png)
